In [210]:
import numpy as np
import pandas as pd
import matplotlib
pd.set_option("display.precision", 4)

from internalForces import BendingMoment, ShearForce
from loadMatrix import LoadMatrix
from memberStiffnessMatrix import MemberStiffnessMatrix
from transformationMatrices import rotationMatrix, transmissionMatrix
from elasticModulus import elasticModulus
from momentOfInertia import momentOfInertia

### Vstupy

In [211]:
# Section properties
width = 200 #mm
height = 300 #mm
concrete = "C30/37"
# -------------------------
E = elasticModulus(concrete)
I = momentOfInertia(width, height)
AREA = width*height*10**-6
print("E = ", E, "kPa")
print("I = ", I, "m-4")
print("A = ", AREA, "m2")

E =  33000000.0 kPa
I =  0.00044999999999999993 m-4
A =  0.06 m2


In [212]:
# lengths
A = 3 #m
B = 3 #m
C = 3 #m

# loads
F = 40 #kN
Q = 5 #kN/m

In [ ]:
# Boundary conditions
node_1 = [0,0,0]
node_2 = [1,1,1]
node_3 = [1,1,1]
node_4 = [0,0,1]

In [213]:
%%HTML
<img src="./assets/schema.png" width="400" height="400" style="position: relative"/>

### Vzorové riešenie na prúte 1-2

In [214]:
k12 = MemberStiffnessMatrix.localMember(C,AREA,E,I)
k12

,0,1,2
0,660000.0,0.0,0.0
1,0.0,6600.0,9900.0
2,0.0,9900.0,19800.0


In [215]:
alfa = 270 #deg
A0 = rotationMatrix(alfa)
A0

,0,1,2
0,-1.8370e-16,-1.0000e+00,0
1,1.0000e+00,-1.8370e-16,0
2,0.0000e+00,0.0000e+00,1


In [216]:
lx = 0 #m
ly = -3 #m

B0 = transmissionMatrix(lx,ly)
B0

,0,1,2
0,-1,0,0
1,0,-1,0
2,3,0,-1


In [217]:
k1 = MemberStiffnessMatrix.globalMember(A0, k12, B0)
k1

,0,1,2,0,1,2
0,6.6000e+03,1.2003e-10,9.9000e+03,-6.6000e+03,-1.2003e-10,9.9000e+03
1,1.2003e-10,6.6000e+05,-1.8186e-12,-1.2003e-10,-6.6000e+05,3.6190e-10
2,9.9000e+03,-1.8186e-12,1.9800e+04,-9.9000e+03,1.8186e-12,9.9000e+03
0,-6.6000e+03,-1.2003e-10,-9.9000e+03,6.6000e+03,1.2003e-10,-9.9000e+03
1,-1.2003e-10,-6.6000e+05,1.8186e-12,1.2003e-10,6.6000e+05,-3.6190e-10
2,9.9000e+03,3.6190e-10,9.9000e+03,-9.9000e+03,-3.6190e-10,1.9800e+04


$$F = [N,V,M]

Primárne vnútorné sily v bode - ***0***

In [218]:
N_11 = 0 #kN
V_11 = ShearForce.distributedLoad(C,Q) #kN
M_11 = BendingMoment.distributedLoad_A(C,Q) #kNm

print("N_11 = ", N_11, "kN")
print("V_11 = ", V_11, "kN")
print("M_11 = ", M_11, "kNm")

N_11 =  0 kN
V_11 =  7.5 kN
M_11 =  -3.75 kNm


Primárne vnútorné sily v bode - ***1***

In [219]:
N_21 = 0 #kN
V_21 = ShearForce.distributedLoad(C,Q) #kN
M_21 = BendingMoment.distributedLoad_A(C,Q) #kNm

print("N_21 = ", N_21, "kN")
print("V_21 = ", V_21, "kN")
print("M_21 = ", M_21, "kNm")

N_21 =  0 kN
V_21 =  7.5 kN
M_21 =  -3.75 kNm


Silová matica v LSS

In [220]:
F1_l = LoadMatrix.localMatrix(N_11,V_11,M_11)
F1_l

,0
0,0.00
1,7.50
2,-3.75


In [221]:
F2_l = LoadMatrix.localMatrix(N_21,V_21,M_21)
F2_l

,0
0,0.00
1,7.50
2,-3.75


Silová matica v GSS

In [222]:
F1_g = LoadMatrix.globalMatrix(F1_l, A0.T)
F2_g = LoadMatrix.globalMatrix(F2_l, A0.T)

F12 = LoadMatrix.superMatrix(F1_g, F2_g)
F12

,0
0,7.5000e+00
1,-1.3777e-15
2,-3.7500e+00
0,7.5000e+00
1,-1.3777e-15
2,-3.7500e+00


### Riešenie na prúte 2-3

In [223]:
alfa = 0 #deg
lx = 3 #m
ly = 0 #m

In [224]:
k23 = MemberStiffnessMatrix.localMember(B,AREA,E,I)
A0 = rotationMatrix(alfa)
B0 = transmissionMatrix(lx,ly)
k2 = MemberStiffnessMatrix.globalMember(A0, k23, B0)
k2

,0,1,2,0,1,2
0,660000.0,0.0,0.0,-660000.0,0.0,0.0
1,0.0,6600.0,9900.0,0.0,-6600.0,9900.0
2,0.0,9900.0,19800.0,0.0,-9900.0,9900.0
0,-660000.0,0.0,0.0,660000.0,0.0,0.0
1,0.0,-6600.0,-9900.0,0.0,6600.0,-9900.0
2,0.0,9900.0,9900.0,0.0,-9900.0,19800.0


In [225]:
N_22 = 0 #kN
V_22 = ShearForce.pointLoad(F) #kN
M_22 = BendingMoment.pointLoad_A(B, F) #kNm
N_32 = 0 #kN
V_32 = ShearForce.pointLoad(F) #kN
M_32 = BendingMoment.pointLoad_B(B, F) #kNm

In [226]:
F2_l = LoadMatrix.localMatrix(N_22,V_22,M_22)
F3_l = LoadMatrix.localMatrix(N_32,V_32,M_32)
F2_g = LoadMatrix.globalMatrix(F2_l, A0.T)
F3_g = LoadMatrix.globalMatrix(F3_l, A0.T)

F23 = LoadMatrix.superMatrix(F2_g, F3_g)
F23

,0
0,0.0
1,20.0
2,15.0
0,0.0
1,20.0
2,-15.0


### Riešenie na prúte 3-4

In [227]:
alfa = 90 #deg
lx = 0 #m
ly = 3 #m

In [228]:
k34 = MemberStiffnessMatrix.localMember(B,AREA,E,I)
A0 = rotationMatrix(alfa)
B0 = transmissionMatrix(lx,ly)
k3 = MemberStiffnessMatrix.globalMember(A0, k34, B0)
k3

,0,1,2,0,1,2
0,6.6000e+03,4.0009e-11,-9.9000e+03,-6.6000e+03,-4.0009e-11,-9.9000e+03
1,4.0009e-11,6.6000e+05,6.0620e-13,-4.0009e-11,-6.6000e+05,-1.2063e-10
2,-9.9000e+03,6.0620e-13,1.9800e+04,9.9000e+03,-6.0620e-13,9.9000e+03
0,-6.6000e+03,-4.0009e-11,9.9000e+03,6.6000e+03,4.0009e-11,9.9000e+03
1,-4.0009e-11,-6.6000e+05,-6.0620e-13,4.0009e-11,6.6000e+05,1.2063e-10
2,-9.9000e+03,-1.2063e-10,9.9000e+03,9.9000e+03,1.2063e-10,1.9800e+04


In [229]:
N_3 = 0 #kN
V_3 = 0 #kN
M_3 = 0 #kNm
N_4 = 0 #kN
V_4 = 0 #kN
M_4 = 0 #kNm

In [230]:
F3_l = LoadMatrix.localMatrix(N_3,V_3,M_3)
F4_l = LoadMatrix.localMatrix(N_4,V_4,M_4)
F3_g = LoadMatrix.globalMatrix(F3_l, A0.T)
F4_g = LoadMatrix.globalMatrix(F4_l, A0.T)

F34 = LoadMatrix.superMatrix(F3_g, F4_g)
F34

,0
0,0.0
1,0.0
2,0.0
0,0.0
1,0.0
2,0.0


In [231]:
zeros = pd.DataFrame(np.zeros((12, 12)))
zeros

,0,1,2,3,4,5,6,7,8,9,10,11
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [232]:
k1.columns=[0,1,2,3,4,5]
k1 = k1.reset_index(drop=True)
k1

,0,1,2,3,4,5
0,6.6000e+03,1.2003e-10,9.9000e+03,-6.6000e+03,-1.2003e-10,9.9000e+03
1,1.2003e-10,6.6000e+05,-1.8186e-12,-1.2003e-10,-6.6000e+05,3.6190e-10
2,9.9000e+03,-1.8186e-12,1.9800e+04,-9.9000e+03,1.8186e-12,9.9000e+03
3,-6.6000e+03,-1.2003e-10,-9.9000e+03,6.6000e+03,1.2003e-10,-9.9000e+03
4,-1.2003e-10,-6.6000e+05,1.8186e-12,1.2003e-10,6.6000e+05,-3.6190e-10
5,9.9000e+03,3.6190e-10,9.9000e+03,-9.9000e+03,-3.6190e-10,1.9800e+04


In [233]:
layer1 = zeros.add(k1,fill_value=0)
layer1

,0,1,2,3,4,5,6,7,8,9,10,11
0,6.6000e+03,1.2003e-10,9.9000e+03,-6.6000e+03,-1.2003e-10,9.9000e+03,0.0,0.0,0.0,0.0,0.0,0.0
1,1.2003e-10,6.6000e+05,-1.8186e-12,-1.2003e-10,-6.6000e+05,3.6190e-10,0.0,0.0,0.0,0.0,0.0,0.0
2,9.9000e+03,-1.8186e-12,1.9800e+04,-9.9000e+03,1.8186e-12,9.9000e+03,0.0,0.0,0.0,0.0,0.0,0.0
3,-6.6000e+03,-1.2003e-10,-9.9000e+03,6.6000e+03,1.2003e-10,-9.9000e+03,0.0,0.0,0.0,0.0,0.0,0.0
4,-1.2003e-10,-6.6000e+05,1.8186e-12,1.2003e-10,6.6000e+05,-3.6190e-10,0.0,0.0,0.0,0.0,0.0,0.0
5,9.9000e+03,3.6190e-10,9.9000e+03,-9.9000e+03,-3.6190e-10,1.9800e+04,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,0.0,0.0,0.0,0.0,0.0,0.0


In [234]:
k2.columns=[3,4,5,6,7,8]
k2 = k2.reset_index(drop=True).rename(index={0: 3, 1: 4, 2: 5, 3: 6, 4: 7, 5: 8})
k2

,3,4,5,6,7,8
3,660000.0,0.0,0.0,-660000.0,0.0,0.0
4,0.0,6600.0,9900.0,0.0,-6600.0,9900.0
5,0.0,9900.0,19800.0,0.0,-9900.0,9900.0
6,-660000.0,0.0,0.0,660000.0,0.0,0.0
7,0.0,-6600.0,-9900.0,0.0,6600.0,-9900.0
8,0.0,9900.0,9900.0,0.0,-9900.0,19800.0


In [235]:
layer2 = layer1.add(k2,fill_value=0)
layer2

,0,1,2,3,4,5,6,7,8,9,10,11
0,6.6000e+03,1.2003e-10,9.9000e+03,-6.6000e+03,-1.2003e-10,9.9000e+03,0.0,0.0,0.0,0.0,0.0,0.0
1,1.2003e-10,6.6000e+05,-1.8186e-12,-1.2003e-10,-6.6000e+05,3.6190e-10,0.0,0.0,0.0,0.0,0.0,0.0
2,9.9000e+03,-1.8186e-12,1.9800e+04,-9.9000e+03,1.8186e-12,9.9000e+03,0.0,0.0,0.0,0.0,0.0,0.0
3,-6.6000e+03,-1.2003e-10,-9.9000e+03,6.6660e+05,1.2003e-10,-9.9000e+03,-660000.0,0.0,0.0,0.0,0.0,0.0
4,-1.2003e-10,-6.6000e+05,1.8186e-12,1.2003e-10,6.6660e+05,9.9000e+03,0.0,-6600.0,9900.0,0.0,0.0,0.0
5,9.9000e+03,3.6190e-10,9.9000e+03,-9.9000e+03,9.9000e+03,3.9600e+04,0.0,-9900.0,9900.0,0.0,0.0,0.0
6,0.0000e+00,0.0000e+00,0.0000e+00,-6.6000e+05,0.0000e+00,0.0000e+00,660000.0,0.0,0.0,0.0,0.0,0.0
7,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,-6.6000e+03,-9.9000e+03,0.0,6600.0,-9900.0,0.0,0.0,0.0
8,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,9.9000e+03,9.9000e+03,0.0,-9900.0,19800.0,0.0,0.0,0.0
9,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,0.0,0.0,0.0,0.0,0.0,0.0


In [236]:
k3.columns=[6,7,8,9,10,11]
k3 = k3.reset_index(drop=True).rename(index={0: 6, 1: 7, 2: 8, 3: 9, 4: 10, 5: 11})
k3

,6,7,8,9,10,11
6,6.6000e+03,4.0009e-11,-9.9000e+03,-6.6000e+03,-4.0009e-11,-9.9000e+03
7,4.0009e-11,6.6000e+05,6.0620e-13,-4.0009e-11,-6.6000e+05,-1.2063e-10
8,-9.9000e+03,6.0620e-13,1.9800e+04,9.9000e+03,-6.0620e-13,9.9000e+03
9,-6.6000e+03,-4.0009e-11,9.9000e+03,6.6000e+03,4.0009e-11,9.9000e+03
10,-4.0009e-11,-6.6000e+05,-6.0620e-13,4.0009e-11,6.6000e+05,1.2063e-10
11,-9.9000e+03,-1.2063e-10,9.9000e+03,9.9000e+03,1.2063e-10,1.9800e+04


In [237]:
layer3 = layer2.add(k3,fill_value=0)
layer3

,0,1,2,3,4,5,6,7,8,9,10,11
0,6.6000e+03,1.2003e-10,9.9000e+03,-6.6000e+03,-1.2003e-10,9.9000e+03,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00
1,1.2003e-10,6.6000e+05,-1.8186e-12,-1.2003e-10,-6.6000e+05,3.6190e-10,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00
2,9.9000e+03,-1.8186e-12,1.9800e+04,-9.9000e+03,1.8186e-12,9.9000e+03,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00
3,-6.6000e+03,-1.2003e-10,-9.9000e+03,6.6660e+05,1.2003e-10,-9.9000e+03,-6.6000e+05,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00
4,-1.2003e-10,-6.6000e+05,1.8186e-12,1.2003e-10,6.6660e+05,9.9000e+03,0.0000e+00,-6.6000e+03,9.9000e+03,0.0000e+00,0.0000e+00,0.0000e+00
5,9.9000e+03,3.6190e-10,9.9000e+03,-9.9000e+03,9.9000e+03,3.9600e+04,0.0000e+00,-9.9000e+03,9.9000e+03,0.0000e+00,0.0000e+00,0.0000e+00
6,0.0000e+00,0.0000e+00,0.0000e+00,-6.6000e+05,0.0000e+00,0.0000e+00,6.6660e+05,4.0009e-11,-9.9000e+03,-6.6000e+03,-4.0009e-11,-9.9000e+03
7,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,-6.6000e+03,-9.9000e+03,4.0009e-11,6.6660e+05,-9.9000e+03,-4.0009e-11,-6.6000e+05,-1.2063e-10
8,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,9.9000e+03,9.9000e+03,-9.9000e+03,-9.9000e+03,3.9600e+04,9.9000e+03,-6.0620e-13,9.9000e+03
9,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,-6.6000e+03,-4.0009e-11,9.9000e+03,6.6000e+03,4.0009e-11,9.9000e+03


In [238]:
def highlightNotZero(x):
    if x < 0:
        #red
        color = "#ff9999"
    elif x > 0:
        #green
        color = "#ccffcc"
    else:
        color = "#f2f2f2"
    return f"background: {color}"

def parseZeroValues(x):
    if x != 0:
        return '{:.3e}'
    else:
        return '{:.0f}'


In [239]:
layer3.style\
    .format('{:.3e}')\
    .applymap(highlightNotZero)\
    .set_table_styles([{"selector" : "", "props" : [("font-family", "arial")]}])


,0,1,2,3,4,5,6,7,8,9,10,11
0,6.600e+03,1.200e-10,9.900e+03,-6.600e+03,-1.200e-10,9.900e+03,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00
1,1.200e-10,6.600e+05,-1.819e-12,-1.200e-10,-6.600e+05,3.619e-10,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00
2,9.900e+03,-1.819e-12,1.980e+04,-9.900e+03,1.819e-12,9.900e+03,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00
3,-6.600e+03,-1.200e-10,-9.900e+03,6.666e+05,1.200e-10,-9.900e+03,-6.600e+05,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00
4,-1.200e-10,-6.600e+05,1.819e-12,1.200e-10,6.666e+05,9.900e+03,0.000e+00,-6.600e+03,9.900e+03,0.000e+00,0.000e+00,0.000e+00
5,9.900e+03,3.619e-10,9.900e+03,-9.900e+03,9.900e+03,3.960e+04,0.000e+00,-9.900e+03,9.900e+03,0.000e+00,0.000e+00,0.000e+00
6,0.000e+00,0.000e+00,0.000e+00,-6.600e+05,0.000e+00,0.000e+00,6.666e+05,4.001e-11,-9.900e+03,-6.600e+03,-4.001e-11,-9.900e+03
7,0.000e+00,0.000e+00,0.000e+00,0.000e+00,-6.600e+03,-9.900e+03,4.001e-11,6.666e+05,-9.900e+03,-4.001e-11,-6.600e+05,-1.206e-10
8,0.000e+00,0.000e+00,0.000e+00,0.000e+00,9.900e+03,9.900e+03,-9.900e+03,-9.900e+03,3.960e+04,9.900e+03,-6.062e-13,9.900e+03
9,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,-6.600e+03,-4.001e-11,9.900e+03,6.600e+03,4.001e-11,9.900e+03


Matica zohľadňujúca okrajové podmienky

In [241]:
F12 = F12.reset_index(drop=True)
F12

,0
0,7.5000e+00
1,-1.3777e-15
2,-3.7500e+00
3,7.5000e+00
4,-1.3777e-15
5,-3.7500e+00


In [242]:
F23 = F23.reset_index(drop=True).rename(index={0: 3, 1: 4, 2: 5, 3: 6, 4: 7, 5: 8})
F23

,0
3,0.0
4,20.0
5,15.0
6,0.0
7,20.0
8,-15.0


In [243]:
Fx = F12.add(F23, fill_value=0)
Fx

,0
0,7.5000e+00
1,-1.3777e-15
2,-3.7500e+00
3,7.5000e+00
4,2.0000e+01
5,1.1250e+01
6,0.0000e+00
7,2.0000e+01
8,-1.5000e+01


In [244]:
F34 = F34.reset_index(drop=True).rename(index={0: 6, 1: 7, 2: 8, 3: 9, 4: 10, 5: 11})
F34

,0
6,0.0
7,0.0
8,0.0
9,0.0
10,0.0
11,0.0


In [245]:
loadVector = Fx.add(F34, fill_value=0)
loadVector

,0
0,7.5000e+00
1,-1.3777e-15
2,-3.7500e+00
3,7.5000e+00
4,2.0000e+01
5,1.1250e+01
6,0.0000e+00
7,2.0000e+01
8,-1.5000e+01
9,0.0000e+00


In [246]:
boundaryConditions

,0,1,2
0,0,0,0
1,1,1,1
2,1,1,1
3,0,0,1


In [247]:
node_1 = [0,0,0]
node_2 = [1,1,1]
node_3 = [1,1,1]
node_4 = [0,0,1]

In [248]:
bc = np.array([node_1, node_2, node_3, node_4])
bc

array([[0, 0, 0],
       [1, 1, 1],
       [1, 1, 1],
       [0, 0, 1]])

In [249]:
bc = bc.flatten()
bc

array([0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1])

In [250]:
bc = pd.DataFrame(bc)
bc

,0
0,0
1,0
2,0
3,1
4,1
5,1
6,1
7,1
8,1
9,0


In [251]:
test = bc.loc[(bc!=1).any(axis=1)]
test

,0
0,0
1,0
2,0
9,0
10,0


In [252]:
loadVector

,0
0,7.5000e+00
1,-1.3777e-15
2,-3.7500e+00
3,7.5000e+00
4,2.0000e+01
5,1.1250e+01
6,0.0000e+00
7,2.0000e+01
8,-1.5000e+01
9,0.0000e+00


In [253]:
dfb = bc[bc[0]==0].index.values.astype(int)
dfb

array([ 0,  1,  2,  9, 10])

In [254]:
loadVector.drop(dfb, inplace=True)

In [255]:
loadVector

,0
3,7.50
4,20.00
5,11.25
6,0.00
7,20.00
8,-15.00
11,0.00


In [256]:
dfc = bc[bc[0]==1].index.values.astype(int)
dfc

array([ 3,  4,  5,  6,  7,  8, 11])

In [257]:
layer3.drop(dfb, inplace=True)
modelMatrix = layer3[dfc]
modelMatrix

,3,4,5,6,7,8,11
3,6.6660e+05,1.2003e-10,-9900.0,-6.6000e+05,0.0000e+00,0.0,0.0000e+00
4,1.2003e-10,6.6660e+05,9900.0,0.0000e+00,-6.6000e+03,9900.0,0.0000e+00
5,-9.9000e+03,9.9000e+03,39600.0,0.0000e+00,-9.9000e+03,9900.0,0.0000e+00
6,-6.6000e+05,0.0000e+00,0.0,6.6660e+05,4.0009e-11,-9900.0,-9.9000e+03
7,0.0000e+00,-6.6000e+03,-9900.0,4.0009e-11,6.6660e+05,-9900.0,-1.2063e-10
8,0.0000e+00,9.9000e+03,9900.0,-9.9000e+03,-9.9000e+03,39600.0,9.9000e+03
11,0.0000e+00,0.0000e+00,0.0,-9.9000e+03,-1.2063e-10,9900.0,1.9800e+04


In [258]:
df_inv = pd.DataFrame(np.linalg.pinv(modelMatrix.values), modelMatrix.columns, modelMatrix.index)
df_inv

,3,4,5,6,7,8,11
3,1.8004e-04,-8.2434e-07,4.1878e-05,1.7970e-04,8.2434e-07,1.4177e-05,8.2760e-05
4,-8.2434e-07,1.5134e-06,-4.8091e-07,-8.2537e-07,1.7172e-09,-4.1243e-07,-2.0647e-07
5,4.1878e-05,-4.8091e-07,3.7049e-05,4.1741e-05,4.8091e-07,-4.3477e-06,2.3044e-05
6,1.7970e-04,-8.2537e-07,4.1741e-05,1.8087e-04,8.2537e-07,1.4384e-05,8.3242e-05
7,8.2434e-07,1.7172e-09,4.8091e-07,8.2537e-07,1.5134e-06,4.1243e-07,2.0647e-07
8,1.4177e-05,-4.1243e-07,-4.3477e-06,1.4384e-05,4.1243e-07,3.2393e-05,-9.0044e-06
11,8.2760e-05,-2.0647e-07,2.3044e-05,8.3242e-05,2.0647e-07,-9.0044e-06,9.6628e-05


In [259]:
displacement = df_inv.dot(loadVector)
displacement

,0
3,1.6088e-03
4,2.4897e-05
5,7.9611e-04
6,1.6016e-03
7,3.5709e-05
8,-4.2848e-04
11,1.0150e-03
